# <center><h1>Companhia Aberta Demonstrativo Financeiro</h1></center>

# <h2>Load Libraries</h2>

In [347]:
import pandas as pd
import numpy as np

#<h2>Import Data</h2>

<h3>Load .csv</h3>

In [408]:
cia_aberta_df = pd.read_csv(r'dfp_cia_aberta_BPA_con_2020.csv', encoding='ISO-8859-1', sep=";")    # leio o .csv com os dados de todas as companhias abertas

<h3>Select Company</h3>

In [409]:
cia_aberta_df = cia_aberta_df[cia_aberta_df['CNPJ_CIA'] == '97.837.181/0001-47']    # dataframe = linhas do dataframe onde ('CNPJ_CIA' == '97.837.181/0001-47') == True
                                                                                    # (seleciono somente as linhas relativas a uma companhia de interesse)

In [410]:
df = cia_aberta_df[['ORDEM_EXERC','CD_CONTA','DS_CONTA','VL_CONTA']].copy()
df = df[df['ORDEM_EXERC'] == 'ÚLTIMO']                                         # seleciono somente o último ou penúltimo exercício
df.reset_index(inplace=True, drop=True)

In [411]:
df

,ORDEM_EXERC,CD_CONTA,DS_CONTA,VL_CONTA
0,ÚLTIMO,1,Ativo Total,11498520.0
1,ÚLTIMO,1.01,Ativo Circulante,4220022.0
2,ÚLTIMO,1.01.01,Caixa e Equivalentes de Caixa,1728413.0
3,ÚLTIMO,1.01.02,Aplicações Financeiras,0.0
4,ÚLTIMO,1.01.02.01,Aplicações Financeiras Avaliadas a Valor Justo...,0.0
...,...,...,...,...
71,ÚLTIMO,1.02.04.02.07,Goodwill na aquisição da Caetex Florestal,8767.0
72,ÚLTIMO,1.02.04.02.08,Goodwill na aquisição da Cerâmica Urussanga em...,92944.0
73,ÚLTIMO,1.02.04.02.09,Goodwill na aquisição da Massima Revestimentos...,6110.0
74,ÚLTIMO,1.02.04.02.10,Goodwill na aquisição da Cecrisa Revestimentos...,168430.0


#<h2>Wrangling</h2>

# <h3>Split CD_CONTA</h3>

In [366]:
df['CD_CONTA'].sort_values(ascending=True)

,CD_CONTA
0,1
1,1.01
2,1.01.01
3,1.01.02
4,1.01.02.01
...,...
71,1.02.04.02.07
72,1.02.04.02.08
73,1.02.04.02.09
74,1.02.04.02.10


In [367]:
cd_conta_split = [string.split('.') for string in df['CD_CONTA']]

In [368]:
cd_conta_len = [len(lst) for lst in cd_conta_split]

In [369]:
num_levels = max(cd_conta_len)

In [370]:
num_levels

5

ROUND 1

In [371]:
round=1

JOIN BASATA SULLA LISTA DI LISTE CON I VALORI DEI CODICI

In [372]:
CD_DS_CONTA_dict = {cd: ds for cd, ds in zip(df['CD_CONTA'], df['DS_CONTA'])}

In [373]:
cd_conta_col_1 = [row[round-1] for row in cd_conta_split]

ds_conta_1 = DS_CONTA where cd_conta_len == 1 & CD_CONTA == cd_conta_split[0]

In [379]:
idx_keys = [idx for idx, element in enumerate(cd_conta_len) if element == round]        # round = 1

In [381]:
idx_keys

[0]

In [382]:
keys = [cd_conta_col_1[idx] for idx in idx_keys]

In [383]:
keys

['1']

In [389]:
idx_key_dict = {idx: key for idx in idx_keys for key in keys}

In [390]:
idx_key_dict

{0: '1'}

In [391]:
key_idx_dict = {key: idx for idx in idx_keys for key in keys}

In [392]:
key_idx_dict

{'1': 0}

In [375]:
df['DS_CONTA']

,DS_CONTA
0,Ativo Total
1,Ativo Circulante
2,Caixa e Equivalentes de Caixa
3,Aplicações Financeiras
4,Aplicações Financeiras Avaliadas a Valor Justo...
...,...
71,Goodwill na aquisição da Caetex Florestal
72,Goodwill na aquisição da Cerâmica Urussanga em...
73,Goodwill na aquisição da Massima Revestimentos...
74,Goodwill na aquisição da Cecrisa Revestimentos...


In [376]:
df['DS_CONTA'][0]

'Ativo Total'

ritorna DS_CONTA[idx] dove idx == indice di cd_conta_len == 1
per tutti i cd_conta_split che iniziano per lo stesso codice con len == 1

In [399]:
for key in key_idx_dict:
  print(key_idx_dict[key])

0


In [404]:
for row in cd_conta_split:
  for key in key_idx_dict:
    if row[round-1] == key:
      print(df['DS_CONTA'][key_idx_dict[key]])

Ativo Total
Ativo Total
Ativo Total
Ativo Total
Ativo Total
Ativo Total
Ativo Total
Ativo Total
Ativo Total
Ativo Total
Ativo Total
Ativo Total
Ativo Total
Ativo Total
Ativo Total
Ativo Total
Ativo Total
Ativo Total
Ativo Total
Ativo Total
Ativo Total
Ativo Total
Ativo Total
Ativo Total
Ativo Total
Ativo Total
Ativo Total
Ativo Total
Ativo Total
Ativo Total
Ativo Total
Ativo Total
Ativo Total
Ativo Total
Ativo Total
Ativo Total
Ativo Total
Ativo Total
Ativo Total
Ativo Total
Ativo Total
Ativo Total
Ativo Total
Ativo Total
Ativo Total
Ativo Total
Ativo Total
Ativo Total
Ativo Total
Ativo Total
Ativo Total
Ativo Total
Ativo Total
Ativo Total
Ativo Total
Ativo Total
Ativo Total
Ativo Total
Ativo Total
Ativo Total
Ativo Total
Ativo Total
Ativo Total
Ativo Total
Ativo Total
Ativo Total
Ativo Total
Ativo Total
Ativo Total
Ativo Total
Ativo Total
Ativo Total
Ativo Total
Ativo Total
Ativo Total
Ativo Total


In [405]:
ds_conta_1 = [df['DS_CONTA'][key_idx_dict[key]] if row[round-1] == key else None for row in cd_conta_split for key in key_idx_dict]

['Ativo Total',
 'Ativo Total',
 'Ativo Total',
 'Ativo Total',
 'Ativo Total',
 'Ativo Total',
 'Ativo Total',
 'Ativo Total',
 'Ativo Total',
 'Ativo Total',
 'Ativo Total',
 'Ativo Total',
 'Ativo Total',
 'Ativo Total',
 'Ativo Total',
 'Ativo Total',
 'Ativo Total',
 'Ativo Total',
 'Ativo Total',
 'Ativo Total',
 'Ativo Total',
 'Ativo Total',
 'Ativo Total',
 'Ativo Total',
 'Ativo Total',
 'Ativo Total',
 'Ativo Total',
 'Ativo Total',
 'Ativo Total',
 'Ativo Total',
 'Ativo Total',
 'Ativo Total',
 'Ativo Total',
 'Ativo Total',
 'Ativo Total',
 'Ativo Total',
 'Ativo Total',
 'Ativo Total',
 'Ativo Total',
 'Ativo Total',
 'Ativo Total',
 'Ativo Total',
 'Ativo Total',
 'Ativo Total',
 'Ativo Total',
 'Ativo Total',
 'Ativo Total',
 'Ativo Total',
 'Ativo Total',
 'Ativo Total',
 'Ativo Total',
 'Ativo Total',
 'Ativo Total',
 'Ativo Total',
 'Ativo Total',
 'Ativo Total',
 'Ativo Total',
 'Ativo Total',
 'Ativo Total',
 'Ativo Total',
 'Ativo Total',
 'Ativo Total',
 'Ativo 

In [412]:
df.insert(2, column='DS_CONTA_1', value=ds_conta_1)

In [413]:
df

,ORDEM_EXERC,CD_CONTA,DS_CONTA_1,DS_CONTA,VL_CONTA
0,ÚLTIMO,1,Ativo Total,Ativo Total,11498520.0
1,ÚLTIMO,1.01,Ativo Total,Ativo Circulante,4220022.0
2,ÚLTIMO,1.01.01,Ativo Total,Caixa e Equivalentes de Caixa,1728413.0
3,ÚLTIMO,1.01.02,Ativo Total,Aplicações Financeiras,0.0
4,ÚLTIMO,1.01.02.01,Ativo Total,Aplicações Financeiras Avaliadas a Valor Justo...,0.0
...,...,...,...,...,...
71,ÚLTIMO,1.02.04.02.07,Ativo Total,Goodwill na aquisição da Caetex Florestal,8767.0
72,ÚLTIMO,1.02.04.02.08,Ativo Total,Goodwill na aquisição da Cerâmica Urussanga em...,92944.0
73,ÚLTIMO,1.02.04.02.09,Ativo Total,Goodwill na aquisição da Massima Revestimentos...,6110.0
74,ÚLTIMO,1.02.04.02.10,Ativo Total,Goodwill na aquisição da Cecrisa Revestimentos...,168430.0


DROP ROWS

Seleziona tutte le righe con un solo elemento:

In [414]:
cd_conta_len = [len(lst) for lst in cd_conta_split]

In [415]:
[idx for idx, element in enumerate(cd_conta_len) if element == 1]

[0]

In [416]:
cd_conta_split[0]

['1']

In [417]:
any(row[0] == '1' for row in cd_conta_split[1:])

True

In [418]:
cd_conta_split = cd_conta_split[0+1:]

In [419]:
cd_conta_split

[['1', '01'],
 ['1', '01', '01'],
 ['1', '01', '02'],
 ['1', '01', '02', '01'],
 ['1', '01', '02', '01', '01'],
 ['1', '01', '02', '01', '02'],
 ['1', '01', '02', '02'],
 ['1', '01', '02', '03'],
 ['1', '01', '03'],
 ['1', '01', '03', '01'],
 ['1', '01', '03', '01', '01'],
 ['1', '01', '03', '01', '02'],
 ['1', '01', '03', '02'],
 ['1', '01', '03', '02', '01'],
 ['1', '01', '04'],
 ['1', '01', '05'],
 ['1', '01', '06'],
 ['1', '01', '06', '01'],
 ['1', '01', '07'],
 ['1', '01', '08'],
 ['1', '01', '08', '01'],
 ['1', '01', '08', '02'],
 ['1', '01', '08', '03'],
 ['1', '02'],
 ['1', '02', '01'],
 ['1', '02', '01', '01'],
 ['1', '02', '01', '01', '01'],
 ['1', '02', '01', '02'],
 ['1', '02', '01', '03'],
 ['1', '02', '01', '04'],
 ['1', '02', '01', '04', '01'],
 ['1', '02', '01', '04', '02'],
 ['1', '02', '01', '05'],
 ['1', '02', '01', '06'],
 ['1', '02', '01', '07'],
 ['1', '02', '01', '07', '01'],
 ['1', '02', '01', '08'],
 ['1', '02', '01', '09'],
 ['1', '02', '01', '09', '01'],
 ['1

In [420]:
df = df.drop(df.index[0])

In [421]:
df

,ORDEM_EXERC,CD_CONTA,DS_CONTA_1,DS_CONTA,VL_CONTA
1,ÚLTIMO,1.01,Ativo Total,Ativo Circulante,4220022.0
2,ÚLTIMO,1.01.01,Ativo Total,Caixa e Equivalentes de Caixa,1728413.0
3,ÚLTIMO,1.01.02,Ativo Total,Aplicações Financeiras,0.0
4,ÚLTIMO,1.01.02.01,Ativo Total,Aplicações Financeiras Avaliadas a Valor Justo...,0.0
5,ÚLTIMO,1.01.02.01.01,Ativo Total,Títulos para Negociação,0.0
...,...,...,...,...,...
71,ÚLTIMO,1.02.04.02.07,Ativo Total,Goodwill na aquisição da Caetex Florestal,8767.0
72,ÚLTIMO,1.02.04.02.08,Ativo Total,Goodwill na aquisição da Cerâmica Urussanga em...,92944.0
73,ÚLTIMO,1.02.04.02.09,Ativo Total,Goodwill na aquisição da Massima Revestimentos...,6110.0
74,ÚLTIMO,1.02.04.02.10,Ativo Total,Goodwill na aquisição da Cecrisa Revestimentos...,168430.0


ROUND 2

In [423]:
round=2

In [422]:
cd_conta_split

[['1', '01'],
 ['1', '01', '01'],
 ['1', '01', '02'],
 ['1', '01', '02', '01'],
 ['1', '01', '02', '01', '01'],
 ['1', '01', '02', '01', '02'],
 ['1', '01', '02', '02'],
 ['1', '01', '02', '03'],
 ['1', '01', '03'],
 ['1', '01', '03', '01'],
 ['1', '01', '03', '01', '01'],
 ['1', '01', '03', '01', '02'],
 ['1', '01', '03', '02'],
 ['1', '01', '03', '02', '01'],
 ['1', '01', '04'],
 ['1', '01', '05'],
 ['1', '01', '06'],
 ['1', '01', '06', '01'],
 ['1', '01', '07'],
 ['1', '01', '08'],
 ['1', '01', '08', '01'],
 ['1', '01', '08', '02'],
 ['1', '01', '08', '03'],
 ['1', '02'],
 ['1', '02', '01'],
 ['1', '02', '01', '01'],
 ['1', '02', '01', '01', '01'],
 ['1', '02', '01', '02'],
 ['1', '02', '01', '03'],
 ['1', '02', '01', '04'],
 ['1', '02', '01', '04', '01'],
 ['1', '02', '01', '04', '02'],
 ['1', '02', '01', '05'],
 ['1', '02', '01', '06'],
 ['1', '02', '01', '07'],
 ['1', '02', '01', '07', '01'],
 ['1', '02', '01', '08'],
 ['1', '02', '01', '09'],
 ['1', '02', '01', '09', '01'],
 ['1

In [428]:
df

,ORDEM_EXERC,CD_CONTA,DS_CONTA_1,DS_CONTA,VL_CONTA
1,ÚLTIMO,1.01,Ativo Total,Ativo Circulante,4220022.0
2,ÚLTIMO,1.01.01,Ativo Total,Caixa e Equivalentes de Caixa,1728413.0
3,ÚLTIMO,1.01.02,Ativo Total,Aplicações Financeiras,0.0
4,ÚLTIMO,1.01.02.01,Ativo Total,Aplicações Financeiras Avaliadas a Valor Justo...,0.0
5,ÚLTIMO,1.01.02.01.01,Ativo Total,Títulos para Negociação,0.0
...,...,...,...,...,...
71,ÚLTIMO,1.02.04.02.07,Ativo Total,Goodwill na aquisição da Caetex Florestal,8767.0
72,ÚLTIMO,1.02.04.02.08,Ativo Total,Goodwill na aquisição da Cerâmica Urussanga em...,92944.0
73,ÚLTIMO,1.02.04.02.09,Ativo Total,Goodwill na aquisição da Massima Revestimentos...,6110.0
74,ÚLTIMO,1.02.04.02.10,Ativo Total,Goodwill na aquisição da Cecrisa Revestimentos...,168430.0


In [434]:
cd_conta_col_2 = [row[round-1] for row in cd_conta_split]

In [447]:
idx_keys = [idx for idx, element in enumerate(cd_conta_len) if element == round]

In [448]:
idx_keys

[1, 24]

In [449]:
keys = [cd_conta_col_2[idx] for idx in idx_keys]

In [450]:
keys

['01', '02']

In [451]:
key_idx_dict = {key: idx for idx in idx_keys for key in keys}

In [452]:
key_idx_dict

{'01': 24, '02': 24}

In [205]:
df.insert(3, column='DS_CONTA_2', value=ds_conta_2)

In [206]:
df

,ORDEM_EXERC,CD_CONTA,DS_CONTA_1,DS_CONTA_2,DS_CONTA,VL_CONTA
56648,ÚLTIMO,01,Ativo Total,Ativo Circulante,Ativo Circulante,4220022.0
56650,ÚLTIMO,01.01,Ativo Total,Ativo Circulante,Caixa e Equivalentes de Caixa,1728413.0
56652,ÚLTIMO,01.02,Ativo Total,Ativo Circulante,Aplicações Financeiras,0.0
56654,ÚLTIMO,01.02.01,Ativo Total,Ativo Circulante,Aplicações Financeiras Avaliadas a Valor Justo...,0.0
56656,ÚLTIMO,01.02.01.01,Ativo Total,Ativo Circulante,Títulos para Negociação,0.0
...,...,...,...,...,...,...
56788,ÚLTIMO,02.04.02.07,Ativo Total,Ativo Não Circulante,Goodwill na aquisição da Caetex Florestal,8767.0
56790,ÚLTIMO,02.04.02.08,Ativo Total,Ativo Não Circulante,Goodwill na aquisição da Cerâmica Urussanga em...,92944.0
56792,ÚLTIMO,02.04.02.09,Ativo Total,Ativo Não Circulante,Goodwill na aquisição da Massima Revestimentos...,6110.0
56794,ÚLTIMO,02.04.02.10,Ativo Total,Ativo Não Circulante,Goodwill na aquisição da Cecrisa Revestimentos...,168430.0


Drop Rows

In [216]:
cd_conta_split_2

[['01'],
 ['01', '01'],
 ['01', '02'],
 ['01', '02', '01'],
 ['01', '02', '01', '01'],
 ['01', '02', '01', '02'],
 ['01', '02', '02'],
 ['01', '02', '03'],
 ['01', '03'],
 ['01', '03', '01'],
 ['01', '03', '01', '01'],
 ['01', '03', '01', '02'],
 ['01', '03', '02'],
 ['01', '03', '02', '01'],
 ['01', '04'],
 ['01', '05'],
 ['01', '06'],
 ['01', '06', '01'],
 ['01', '07'],
 ['01', '08'],
 ['01', '08', '01'],
 ['01', '08', '02'],
 ['01', '08', '03'],
 ['02'],
 ['02', '01'],
 ['02', '01', '01'],
 ['02', '01', '01', '01'],
 ['02', '01', '02'],
 ['02', '01', '03'],
 ['02', '01', '04'],
 ['02', '01', '04', '01'],
 ['02', '01', '04', '02'],
 ['02', '01', '05'],
 ['02', '01', '06'],
 ['02', '01', '07'],
 ['02', '01', '07', '01'],
 ['02', '01', '08'],
 ['02', '01', '09'],
 ['02', '01', '09', '01'],
 ['02', '01', '09', '03'],
 ['02', '01', '09', '04'],
 ['02', '01', '10'],
 ['02', '01', '10', '01'],
 ['02', '01', '10', '02'],
 ['02', '01', '10', '03'],
 ['02', '01', '10', '04'],
 ['02', '01', '1

Seleziona tutte le righe con un solo elemento:

In [221]:
cd_conta_len = [len(row) for row in cd_conta_split_2]

In [222]:
print(cd_conta_len)

[1, 2, 2, 3, 4, 4, 3, 3, 2, 3, 4, 4, 3, 4, 2, 2, 2, 3, 2, 2, 3, 3, 3, 1, 2, 3, 4, 3, 3, 3, 4, 4, 3, 3, 3, 4, 3, 3, 4, 4, 4, 3, 4, 4, 4, 4, 4, 4, 2, 3, 4, 4, 4, 3, 2, 3, 3, 3, 2, 3, 4, 4, 4, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]


In [234]:
[idx for idx, element in enumerate(cd_conta_len) if element == 1]

[0, 23]

In [237]:
[cd_conta_split_2[i] for i in [0, 23]]

[['01'], ['02']]

In [243]:
cd_conta_split_2[0+1:23]

[['01', '01'],
 ['01', '02'],
 ['01', '02', '01'],
 ['01', '02', '01', '01'],
 ['01', '02', '01', '02'],
 ['01', '02', '02'],
 ['01', '02', '03'],
 ['01', '03'],
 ['01', '03', '01'],
 ['01', '03', '01', '01'],
 ['01', '03', '01', '02'],
 ['01', '03', '02'],
 ['01', '03', '02', '01'],
 ['01', '04'],
 ['01', '05'],
 ['01', '06'],
 ['01', '06', '01'],
 ['01', '07'],
 ['01', '08'],
 ['01', '08', '01'],
 ['01', '08', '02'],
 ['01', '08', '03']]

In [241]:
any(row[0] == '01' for row in cd_conta_split_2[0+1:23])

True

In [244]:
cd_conta_split_2[23+1:]

[['02', '01'],
 ['02', '01', '01'],
 ['02', '01', '01', '01'],
 ['02', '01', '02'],
 ['02', '01', '03'],
 ['02', '01', '04'],
 ['02', '01', '04', '01'],
 ['02', '01', '04', '02'],
 ['02', '01', '05'],
 ['02', '01', '06'],
 ['02', '01', '07'],
 ['02', '01', '07', '01'],
 ['02', '01', '08'],
 ['02', '01', '09'],
 ['02', '01', '09', '01'],
 ['02', '01', '09', '03'],
 ['02', '01', '09', '04'],
 ['02', '01', '10'],
 ['02', '01', '10', '01'],
 ['02', '01', '10', '02'],
 ['02', '01', '10', '03'],
 ['02', '01', '10', '04'],
 ['02', '01', '10', '05'],
 ['02', '01', '10', '06'],
 ['02', '02'],
 ['02', '02', '01'],
 ['02', '02', '01', '01'],
 ['02', '02', '01', '04'],
 ['02', '02', '01', '05'],
 ['02', '02', '02'],
 ['02', '03'],
 ['02', '03', '01'],
 ['02', '03', '02'],
 ['02', '03', '03'],
 ['02', '04'],
 ['02', '04', '01'],
 ['02', '04', '01', '01'],
 ['02', '04', '01', '02'],
 ['02', '04', '01', '03'],
 ['02', '04', '02'],
 ['02', '04', '02', '01'],
 ['02', '04', '02', '02'],
 ['02', '04', '0

In [245]:
any(row[0] == '01' for row in cd_conta_split_2[1:23])

True

In [263]:
cd_conta_split_3 = cd_conta_split_2[0+1:23] + cd_conta_split_2[23+1:]

In [264]:
cd_conta_split_3

[['01', '01'],
 ['01', '02'],
 ['01', '02', '01'],
 ['01', '02', '01', '01'],
 ['01', '02', '01', '02'],
 ['01', '02', '02'],
 ['01', '02', '03'],
 ['01', '03'],
 ['01', '03', '01'],
 ['01', '03', '01', '01'],
 ['01', '03', '01', '02'],
 ['01', '03', '02'],
 ['01', '03', '02', '01'],
 ['01', '04'],
 ['01', '05'],
 ['01', '06'],
 ['01', '06', '01'],
 ['01', '07'],
 ['01', '08'],
 ['01', '08', '01'],
 ['01', '08', '02'],
 ['01', '08', '03'],
 ['02', '01'],
 ['02', '01', '01'],
 ['02', '01', '01', '01'],
 ['02', '01', '02'],
 ['02', '01', '03'],
 ['02', '01', '04'],
 ['02', '01', '04', '01'],
 ['02', '01', '04', '02'],
 ['02', '01', '05'],
 ['02', '01', '06'],
 ['02', '01', '07'],
 ['02', '01', '07', '01'],
 ['02', '01', '08'],
 ['02', '01', '09'],
 ['02', '01', '09', '01'],
 ['02', '01', '09', '03'],
 ['02', '01', '09', '04'],
 ['02', '01', '10'],
 ['02', '01', '10', '01'],
 ['02', '01', '10', '02'],
 ['02', '01', '10', '03'],
 ['02', '01', '10', '04'],
 ['02', '01', '10', '05'],
 ['02',

In [258]:
df = df.drop(df.index[[0,23]])

In [265]:
df

,ORDEM_EXERC,CD_CONTA,DS_CONTA_1,DS_CONTA_2,DS_CONTA,VL_CONTA
56650,ÚLTIMO,01.01,Ativo Total,Ativo Circulante,Caixa e Equivalentes de Caixa,1728413.0
56652,ÚLTIMO,01.02,Ativo Total,Ativo Circulante,Aplicações Financeiras,0.0
56654,ÚLTIMO,01.02.01,Ativo Total,Ativo Circulante,Aplicações Financeiras Avaliadas a Valor Justo...,0.0
56656,ÚLTIMO,01.02.01.01,Ativo Total,Ativo Circulante,Títulos para Negociação,0.0
56658,ÚLTIMO,01.02.01.02,Ativo Total,Ativo Circulante,Títulos Designados a Valor Justo,0.0
...,...,...,...,...,...,...
56788,ÚLTIMO,02.04.02.07,Ativo Total,Ativo Não Circulante,Goodwill na aquisição da Caetex Florestal,8767.0
56790,ÚLTIMO,02.04.02.08,Ativo Total,Ativo Não Circulante,Goodwill na aquisição da Cerâmica Urussanga em...,92944.0
56792,ÚLTIMO,02.04.02.09,Ativo Total,Ativo Não Circulante,Goodwill na aquisição da Massima Revestimentos...,6110.0
56794,ÚLTIMO,02.04.02.10,Ativo Total,Ativo Não Circulante,Goodwill na aquisição da Cecrisa Revestimentos...,168430.0


ROUND 3

In [266]:
cd_conta_split_3 = [row[1:] for row in cd_conta_split_3]

In [267]:
cd_conta_split_3

[['01'],
 ['02'],
 ['02', '01'],
 ['02', '01', '01'],
 ['02', '01', '02'],
 ['02', '02'],
 ['02', '03'],
 ['03'],
 ['03', '01'],
 ['03', '01', '01'],
 ['03', '01', '02'],
 ['03', '02'],
 ['03', '02', '01'],
 ['04'],
 ['05'],
 ['06'],
 ['06', '01'],
 ['07'],
 ['08'],
 ['08', '01'],
 ['08', '02'],
 ['08', '03'],
 ['01'],
 ['01', '01'],
 ['01', '01', '01'],
 ['01', '02'],
 ['01', '03'],
 ['01', '04'],
 ['01', '04', '01'],
 ['01', '04', '02'],
 ['01', '05'],
 ['01', '06'],
 ['01', '07'],
 ['01', '07', '01'],
 ['01', '08'],
 ['01', '09'],
 ['01', '09', '01'],
 ['01', '09', '03'],
 ['01', '09', '04'],
 ['01', '10'],
 ['01', '10', '01'],
 ['01', '10', '02'],
 ['01', '10', '03'],
 ['01', '10', '04'],
 ['01', '10', '05'],
 ['01', '10', '06'],
 ['02'],
 ['02', '01'],
 ['02', '01', '01'],
 ['02', '01', '04'],
 ['02', '01', '05'],
 ['02', '02'],
 ['03'],
 ['03', '01'],
 ['03', '02'],
 ['03', '03'],
 ['04'],
 ['04', '01'],
 ['04', '01', '01'],
 ['04', '01', '02'],
 ['04', '01', '03'],
 ['04', '02']

In [269]:
[".".join(row) for row in cd_conta_split_3]

['01',
 '02',
 '02.01',
 '02.01.01',
 '02.01.02',
 '02.02',
 '02.03',
 '03',
 '03.01',
 '03.01.01',
 '03.01.02',
 '03.02',
 '03.02.01',
 '04',
 '05',
 '06',
 '06.01',
 '07',
 '08',
 '08.01',
 '08.02',
 '08.03',
 '01',
 '01.01',
 '01.01.01',
 '01.02',
 '01.03',
 '01.04',
 '01.04.01',
 '01.04.02',
 '01.05',
 '01.06',
 '01.07',
 '01.07.01',
 '01.08',
 '01.09',
 '01.09.01',
 '01.09.03',
 '01.09.04',
 '01.10',
 '01.10.01',
 '01.10.02',
 '01.10.03',
 '01.10.04',
 '01.10.05',
 '01.10.06',
 '02',
 '02.01',
 '02.01.01',
 '02.01.04',
 '02.01.05',
 '02.02',
 '03',
 '03.01',
 '03.02',
 '03.03',
 '04',
 '04.01',
 '04.01.01',
 '04.01.02',
 '04.01.03',
 '04.02',
 '04.02.01',
 '04.02.02',
 '04.02.03',
 '04.02.04',
 '04.02.05',
 '04.02.06',
 '04.02.07',
 '04.02.08',
 '04.02.09',
 '04.02.10',
 '04.02.11']

In [270]:
df['CD_CONTA'] = [".".join(row) for row in cd_conta_split_3]

In [271]:
df

,ORDEM_EXERC,CD_CONTA,DS_CONTA_1,DS_CONTA_2,DS_CONTA,VL_CONTA
56650,ÚLTIMO,01,Ativo Total,Ativo Circulante,Caixa e Equivalentes de Caixa,1728413.0
56652,ÚLTIMO,02,Ativo Total,Ativo Circulante,Aplicações Financeiras,0.0
56654,ÚLTIMO,02.01,Ativo Total,Ativo Circulante,Aplicações Financeiras Avaliadas a Valor Justo...,0.0
56656,ÚLTIMO,02.01.01,Ativo Total,Ativo Circulante,Títulos para Negociação,0.0
56658,ÚLTIMO,02.01.02,Ativo Total,Ativo Circulante,Títulos Designados a Valor Justo,0.0
...,...,...,...,...,...,...
56788,ÚLTIMO,04.02.07,Ativo Total,Ativo Não Circulante,Goodwill na aquisição da Caetex Florestal,8767.0
56790,ÚLTIMO,04.02.08,Ativo Total,Ativo Não Circulante,Goodwill na aquisição da Cerâmica Urussanga em...,92944.0
56792,ÚLTIMO,04.02.09,Ativo Total,Ativo Não Circulante,Goodwill na aquisição da Massima Revestimentos...,6110.0
56794,ÚLTIMO,04.02.10,Ativo Total,Ativo Não Circulante,Goodwill na aquisição da Cecrisa Revestimentos...,168430.0


In [272]:
third_column = [row[0] for row in cd_conta_split_3]

In [273]:
print(third_column)

['01', '02', '02', '02', '02', '02', '02', '03', '03', '03', '03', '03', '03', '04', '05', '06', '06', '07', '08', '08', '08', '08', '01', '01', '01', '01', '01', '01', '01', '01', '01', '01', '01', '01', '01', '01', '01', '01', '01', '01', '01', '01', '01', '01', '01', '01', '02', '02', '02', '02', '02', '02', '03', '03', '03', '03', '04', '04', '04', '04', '04', '04', '04', '04', '04', '04', '04', '04', '04', '04', '04', '04', '04']


In [274]:
CD_DS_CONTA_dict = {cd: ds for cd, ds in zip(df['CD_CONTA'], df['DS_CONTA'])}

In [275]:
def join(key):
    ds_conta = CD_DS_CONTA_dict[key]
    return ds_conta

In [276]:
ds_conta_3 = [join(key) for key in third_column]

In [277]:
ds_conta_3

['Ativo Realizável a Longo Prazo',
 'Investimentos',
 'Investimentos',
 'Investimentos',
 'Investimentos',
 'Investimentos',
 'Investimentos',
 'Imobilizado',
 'Imobilizado',
 'Imobilizado',
 'Imobilizado',
 'Imobilizado',
 'Imobilizado',
 'Intangível',
 'Ativos Biológicos',
 'Tributos a Recuperar',
 'Tributos a Recuperar',
 'Despesas Antecipadas',
 'Outros Ativos Circulantes',
 'Outros Ativos Circulantes',
 'Outros Ativos Circulantes',
 'Outros Ativos Circulantes',
 'Ativo Realizável a Longo Prazo',
 'Ativo Realizável a Longo Prazo',
 'Ativo Realizável a Longo Prazo',
 'Ativo Realizável a Longo Prazo',
 'Ativo Realizável a Longo Prazo',
 'Ativo Realizável a Longo Prazo',
 'Ativo Realizável a Longo Prazo',
 'Ativo Realizável a Longo Prazo',
 'Ativo Realizável a Longo Prazo',
 'Ativo Realizável a Longo Prazo',
 'Ativo Realizável a Longo Prazo',
 'Ativo Realizável a Longo Prazo',
 'Ativo Realizável a Longo Prazo',
 'Ativo Realizável a Longo Prazo',
 'Ativo Realizável a Longo Prazo',
 'At

In [278]:
df.insert(4, column='DS_CONTA_3', value=ds_conta_3)

In [279]:
df

,ORDEM_EXERC,CD_CONTA,DS_CONTA_1,DS_CONTA_2,DS_CONTA_3,DS_CONTA,VL_CONTA
56650,ÚLTIMO,01,Ativo Total,Ativo Circulante,Ativo Realizável a Longo Prazo,Caixa e Equivalentes de Caixa,1728413.0
56652,ÚLTIMO,02,Ativo Total,Ativo Circulante,Investimentos,Aplicações Financeiras,0.0
56654,ÚLTIMO,02.01,Ativo Total,Ativo Circulante,Investimentos,Aplicações Financeiras Avaliadas a Valor Justo...,0.0
56656,ÚLTIMO,02.01.01,Ativo Total,Ativo Circulante,Investimentos,Títulos para Negociação,0.0
56658,ÚLTIMO,02.01.02,Ativo Total,Ativo Circulante,Investimentos,Títulos Designados a Valor Justo,0.0
...,...,...,...,...,...,...,...
56788,ÚLTIMO,04.02.07,Ativo Total,Ativo Não Circulante,Intangível,Goodwill na aquisição da Caetex Florestal,8767.0
56790,ÚLTIMO,04.02.08,Ativo Total,Ativo Não Circulante,Intangível,Goodwill na aquisição da Cerâmica Urussanga em...,92944.0
56792,ÚLTIMO,04.02.09,Ativo Total,Ativo Não Circulante,Intangível,Goodwill na aquisição da Massima Revestimentos...,6110.0
56794,ÚLTIMO,04.02.10,Ativo Total,Ativo Não Circulante,Intangível,Goodwill na aquisição da Cecrisa Revestimentos...,168430.0
